In [ ]:
#!pip install pandas
#!pip install matplotlib
#!pip install numpy
#!pip install librosa

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import librosa.display
import soundfile
import os

from glob import glob

import librosa
import IPython.display as ipd

KeyboardInterrupt: 

## Read Audio Files


In [2]:
audio_files = glob('Audio Data/Audio Data/*/*.wav')


#Playing an audio file using IPython.display
ipd.Audio(audio_files[0])

NameError: name 'glob' is not defined

## Convert to Spectrogram

In [ ]:
def audio_to_spectrogram(file_path, save_image=True):
    # Load the audio file
    y, sr = librosa.load(file_path, sr=None)

    # Compute Short-Time Fourier Transform (STFT)
    D = librosa.stft(y)

    # Convert amplitude to decibels (log scale)
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

    # Plot the spectrogram
    plt.figure(figsize=(10, 5))
    librosa.display.specshow(S_db, sr=sr, x_axis="time", y_axis="log")
    plt.colorbar(label="Decibels (dB)")
    plt.title(f"Spectrogram of {file_path.split('/')[-1]}")
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")

    # Save the spectrogram as an image
    if save_image:
        plt.savefig(file_path.replace(".wav", "_spectrogram.png"))

    plt.show()